In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

# 计算xgboost+BiLSTM
df_xgboost = pd.read_csv("./result/XGBoostpre.csv")
df_BiLSTM = pd.read_csv("./result/BiLSTMpre.csv")

In [8]:
# 计算基模型的累计误差
def CumMSE(df_model:pd.DataFrame):
    """
    df_model第一列为true，第二列为pre
    """
    cum_mse = []
    for i in range(len(df_model)):
        cum_mse.append(mean_squared_error(df_model.iloc[:i+1,0],df_model.iloc[:i+1,1]))
    return cum_mse

cum_mse_xgboost = CumMSE(df_xgboost)
cum_mse_BiLSTM = CumMSE(df_BiLSTM)

# 计算组合权重
def CombineWeight(cum_mse1:list,cum_mse2:list):
    """
    :parameters:
    cum_mse1: 模型1累计误差 
    cum_mse2: 模型2累计误差
    
    :return: 
    模型1的组合权重
    """
    weightList = []
    for i in range(len(cum_mse1)):
        weightOfModel1 = cum_mse1[i]/sum([cum_mse1[i],cum_mse2[i]])
        weightList.append(weightOfModel1)
    return weightList

weight_xgboost = CombineWeight(cum_mse_xgboost,cum_mse_BiLSTM)
    

In [12]:
df_xgboost.loc[:,'y_true'].tolist()

[0.21986437,
 0.20608182,
 0.21198863,
 0.21789543,
 0.21767665,
 0.21461387,
 0.22292715,
 0.23517829,
 0.23583461,
 0.22598994,
 0.23670968,
 0.26646248,
 0.26974404,
 0.2714942,
 0.26996282,
 0.2894334,
 0.31152922,
 0.33143732,
 0.33362502,
 0.33471888,
 0.34675127,
 0.36447167,
 0.38109824,
 0.38569242,
 0.4042879,
 0.43294683,
 0.4532925,
 0.45307374,
 0.450886,
 0.4664187,
 0.48523298,
 0.5178298,
 0.5217677,
 0.5287683,
 0.54036313,
 0.54561365,
 0.56289655,
 0.59243053,
 0.59680593,
 0.5978998,
 0.60643184,
 0.6121199,
 0.6178079,
 0.6385911,
 0.65412384,
 0.66659373,
 0.6563115,
 0.68650186,
 0.69087726,
 0.69262743,
 0.6904397,
 0.70159703,
 0.71756727,
 0.7217239,
 0.72369283,
 0.7160359,
 0.71625465,
 0.72063005,
 0.7180048,
 0.73266244,
 0.7440385,
 0.7337563,
 0.7234741,
 0.7201925,
 0.7147232,
 0.7138482,
 0.7210676,
 0.7223802,
 0.71100414,
 0.69722164,
 0.7024721,
 0.68387663,
 0.68212646,
 0.68190765,
 0.68081385,
 0.6462481,
 0.6506235,
 0.6479983,
 0.6285277,
 0.61

In [13]:
# 计算xgboost+BiLSTM组合预测值
def CombineModel(df_Model1:pd.DataFrame,df_Model2:pd.DataFrame,combineWeightOfModel1:list):
    """
    :parameters:
    df_Model1: 模型1的结果
    df_Model2: 模型2的结果
    combineWeightOfModel1: 模型1的组合权重

    :return: 预测值
    """
    y_pre = []
    for idx,weight in enumerate(combineWeightOfModel1):
        prediction = df_Model1['y_pre'][idx] *weight + df_Model2['y_pre'][idx]*(1-weight)
        y_pre.append(prediction)
    return y_pre

y_pre = CombineModel(df_xgboost,df_BiLSTM,weight_xgboost)
y = df_xgboost.loc[:,'y_true']


In [14]:
# 计算xgboost+BiLSTM组合评价
r2Score = r2_score(y_true=y, y_pred=y_pre)
meanSquaredError = mean_squared_error(y_true=y, y_pred=y_pre)
meanAbsoluteError = mean_absolute_error(y_true=y, y_pred=y_pre)
print("r2Score: ", r2Score)
print("meanSquaredError: ", meanSquaredError)
print('RMSE: ',np.sqrt(meanSquaredError))
print("meanAbsoluteError: ", meanAbsoluteError)


r2Score:  0.9739721594611396
meanSquaredError:  0.0013440158933059987
RMSE:  0.03666082232173739
meanAbsoluteError:  0.024732691685116867
